In [36]:
%matplotlib inline

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pysd
import pyNetLogo
import time
from ema_workbench import (Model, RealParameter, ScalarOutcome, ema_logging,
                           perform_experiments, TimeSeriesOutcome)
from ema_workbench.em_framework.parameters import create_parameters

pd.set_option('display.max_columns', 30000)
pd.set_option('display.max_rows', 30000)

In [55]:
# The aim of this test is to use a simple vensim model twice and to take the average of the outcome
# The simplemodel.mdl simply calculates y = x1*x2-x3
# ematest.xlsx contains two different sheets with different ranges for x1, x2 and x3

tabbladen = ['Blad1', 'Blad2']

testexcel = []
for blad in tabbladen:
    temppd = pd.read_excel('ematest.xlsx', blad)
    testexcel.append(create_parameters(temppd))
    

def defineinput (x1,x2,x3):
    x = 0
    return x
    
inputmodel = Model('inputmodel', function=defineinput)

n_experiments = 10

fullinput = []
for i in range(2):
    inputmodel.uncertainties = testexcel[0]
    singleinput = perform_experiments(inputmodel, n_experiments)
    fullinput.append(singleinput)
    
fullresults = []
for j in range(n_experiments):
    
    x = ['x1', 'x2', 'x3']

    testdict = {}
    for n in range(2):
        singledict = {}
        for var,i in zip(x, range(len(x))):
            singledict[var] = fullinput[n][0][j][i]
        testdict[n] = singledict
    
    simplemodel = pysd.read_vensim("simplemodel.mdl")

    results = []
    for value in testdict.values():
        results.append(simplemodel.run(params=value))
    
    ytotal = 0
    for yresult in results:
        y = yresult['y'][0]
        ytotal += y

    yavg = ytotal / len(results)
    fullresults.append(yavg)

In [61]:
a = {'y': fullresults}
a

{'y': [-0.004309788799375742,
  -0.012083188343999032,
  0.015252740757848517,
  0.021411656105106973,
  0.006323081474131839,
  0.02134971282962175,
  0.0005907733483789136,
  -0.03305381259135524,
  -0.0002940702621004013,
  -0.00998217860509921]}